In [ ]:
#local
#for test
#awk -F',' 'NR > 1 && NR <= 1000 {print $1}' anxiety_fMRI_participants.csv > file_paths1.txt
#awk -F',' 'NR > 48302 {print $1}' anxiety_fMRI_participants.csv > file_paths.txt
#for all
!awk -F',' '{print $1}' gad7_need_data_set.csv > file_paths.txt

In [ ]:
%%bash
# Function: download_imaging_files()
# Description:
#   This function automates the download of imaging files from a RAP project based
#   on paths specified in a txt file.
#
# Parameters:
#   $1 - A txt file containing file paths to bulk imaging files on the RAP
#
# Example Usage:
#  download_imaging_files file_paths.txt

download_imaging_files() {
  # Check if the file_paths.txt argument is provided and exists
  if [[ -z "$1" || ! -f "$1" ]]; then
    echo "Error: file_paths.txt is not provided."
    return 1
  fi

  # Get the project ID
  local PR
  PR=$(dx env | grep project- | cut -f 2)
  
  # Check if the project ID was successfully retrieved
  if [[ -z "$PR" ]]; then
    echo "Error: Could not retrieve project ID."
    return 1
  fi

  # Construct and execute the dx download commands for each line in the file_paths.txt
  local file_paths_file=$1
  local $output_dir=$2
  while IFS= read -r file_path; do
    local command="dx download --lightweight ${PR}:\"${file_path}\" -o $output_dir"
    eval "$command"
  done < "$file_paths_file"
}

In [ ]:
current_dir=$(pwd)
output_dir="${current_dir}/data_structural"
download_imaging_files file_paths.txt $output_dir

In [2]:
import pandas as pd
import os
# Get the current working directory
current_dir = os.getcwd()
df = pd.read_csv("gad7_need_data_set.csv")
df_name = df['filepath'].str.split(r'[/]', expand=True)[5]
data_path = os.path.join(current_dir, 'data_structural/output.txt')
df_name.to_csv(data_path, sep='\t', index=False, header=False)
#only if data did not fully downloaded
#df_name_left=df_name.iloc[48301:,]
#df_name_left.to_csv('output.txt', sep='\t', index=False, header=False)

OSError: Cannot save file into a non-existent directory: '/Users/xiaoqianxiao/PycharmProjects/UKB_structural/data_structural'

In [ ]:
%%bash
# Function: upload_imaging_files()
# Description:
#   This function automates the download of imaging files from a RAP project based
#   on paths specified in a txt file.
#
# Parameters:
#   $1 - A txt file containing file paths to bulk imaging files on the RAP
#
# Example Usage:
#  upload_imaging_files file_paths.txt 

upload_imaging_files() {
  # Check if the file_paths.txt argument is provided and exists
  if [[ -z "$1" || ! -f "$1" ]]; then
    echo "Error: file_paths.txt is not provided."
    return 1
  fi

  # Get the project ID
  local PR
  PR=$(dx env | grep project- | cut -f 2)
  
  # Check if the project ID was successfully retrieved
  if [[ -z "$PR" ]]; then
    echo "Error: Could not retrieve project ID."
    return 1
  fi

  # Construct and execute the dx download commands for each line in the file_paths.txt
  local file_paths_file=$1
  while IFS= read -r file_path; do
    local command="dx upload "${file_path}" --path "/data/""
    eval "$command"
  done < "$file_paths_file"
}

In [ ]:
!upload_imaging_files output.txt